In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5


In [4]:
!pip install -r requirements.txt


In [ ]:
import os

def check_dataset_structure(image_dir, label_dir):
    all_images_exist = True
    for label_file in os.listdir(label_dir):
        image_file = label_file.replace('txt', 'jpg')  # or 'png', depending on your dataset
        image_path = os.path.join(image_dir, image_file)
        if not os.path.exists(image_path):
            print(f"Image file {image_path} does not exist.")
            all_images_exist = False

    if all_images_exist:
        print("All image files exist.")
    else:
        print("Some image files are missing. Please fix the issues above.")

check_dataset_structure('/content/drive/MyDrive/yolo/dataset/images/train', '/content/drive/MyDrive/yolo/dataset/labels/train')
check_dataset_structure('/content/drive/MyDrive/yolo/dataset/images/val', '/content/drive/MyDrive/yolo/dataset/labels/val')


Streaming output truncated to the last 5000 lines.
Image file /content/drive/MyDrive/yolo/dataset/images/train/aa3ee35fa8c74513_yolo_yolo.jpg does not exist.
Image file /content/drive/MyDrive/yolo/dataset/images/train/acb05b6ad8225de0_yolo_yolo.jpg does not exist.
Image file /content/drive/MyDrive/yolo/dataset/images/train/acb0d369183bbde3_yolo_yolo.jpg does not exist.
Image file /content/drive/MyDrive/yolo/dataset/images/train/ae1490735ed4de63_yolo_yolo.jpg does not exist.
Image file /content/drive/MyDrive/yolo/dataset/images/train/af9a783d431b251f_yolo_yolo.jpg does not exist.
Image file /content/drive/MyDrive/yolo/dataset/images/train/afb294edd3fdd66e_yolo_yolo.jpg does not exist.
Image file /content/drive/MyDrive/yolo/dataset/images/train/b0812a7ff7819219_yolo_yolo.jpg does not exist.
Image file /content/drive/MyDrive/yolo/dataset/images/train/f57d7f62ed0f2e7c_yolo.jpg does not exist.
Image file /content/drive/MyDrive/yolo/dataset/images/train/b0e76a52f685ee80_yolo_yolo.jpg does no

In [ ]:
def check_hidden_files(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.startswith('.'):
                print(f"Hidden file found: {os.path.join(root, file)}")
                os.remove(os.path.join(root, file))
                print(f"Deleted hidden file: {os.path.join(root, file)}")
            if not file.endswith(('.jpg', '.png', '.txt')):  # Assuming images are jpg or png
                print(f"Unexpected file format found: {os.path.join(root, file)}")
                os.remove(os.path.join(root, file))
                print(f"Deleted unexpected file: {os.path.join(root, file)}")

check_hidden_files('/content/drive/MyDrive/yolo/dataset/images/train')
check_hidden_files('/content/drive/MyDrive/yolo/dataset/labels/train')


In [ ]:
import pickle

cache_file = '/content/drive/MyDrive/yolo/dataset/labels/train.cache'
with open(cache_file, 'rb') as f:
    cache = pickle.load(f)
    for key, value in cache.items():
        print(f"{key}: {value}")


UnpicklingError: STACK_GLOBAL requires str

In [5]:
%cd /content/yolov5

/content/yolov5


In [ ]:
!python train.py --img 640 --batch 8 --epochs 50 --data /content/drive/MyDrive/yolo/data.yaml --weights yolov5s.pt --cache --save-period 5


2024-07-03 17:01:57.835921: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-03 17:01:57.836011: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-03 17:01:57.838057: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=/content/drive/MyDrive/yolo/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=Fals

In [ ]:
!pip install opencv-python-headless

In [ ]:
import torch
import cv2
from pathlib import Path
from yolov5.utils.general import check_img_size
from yolov5.utils.torch_utils import select_device, time_sync
from yolov5.utils.plots import Annotator, colors
from yolov5.utils.datasets import letterbox

# Load the YOLO model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='runs/train/exp/weights/best.pt')

# Set up webcam
cap = cv2.VideoCapture(0)  # Change the index if you have multiple cameras

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Prepare the frame for the model
    img = letterbox(frame, 640, stride=32, auto=True)[0]
    img = img.transpose((2, 0, 1))[::-1]  # HWC to CHW, BGR to RGB
    img = np.ascontiguousarray(img)

    # Run inference
    results = model(img)

    # Render results
    annotator = Annotator(frame, line_width=2, example=str(names))
    for *xyxy, conf, cls in results:
        c = int(cls)  # integer class
        label = f'{names[c]} {conf:.2f}'
        annotator.box_label(xyxy, label, color=colors(c, True))

    # Display the resulting frame
    cv2.imshow('frame', annotator.result())

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'yolov5.utils.datasets'

training model from sratch


In [ ]:
!python val.py --weights runs/train/exp/weights/best.pt --data /content/drive/MyDrive/yolo/data.yaml --img 640


val: data=/content/drive/MyDrive/yolo/data.yaml, weights=['runs/train/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-334-g100a423b Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7039792 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/drive/MyDrive/yolo/dataset/labels/val.cache... 1019 images, 0 backgrounds, 0 corrupt: 100% 1019/1019 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 32/32 [00:41<00:00,  1.28s/it]
                   all       1019       1477        0.7       0.36      0.412      0.238
                   Axe       1019         22          1          0     0.0496     0.02

In [ ]:
!python detect.py --weights runs/train/exp/weights/best.pt --source /path/to/your/test/images --img 640 --conf 0.25


detect: weights=['runs/train/exp/weights/best.pt'], source=/path/to/your/test/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-334-g100a423b Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7039792 parameters, 0 gradients, 15.8 GFLOPs
Traceback (most recent call last):
  File "/content/yolov5/detect.py", line 313, in <module>
    main(opt)
  File "/content/yolov5/detect.py", line 308, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
 

In [ ]:
!cp runs/train/exp/weights/best.pt /content/drive/MyDrive/yolo/

cp: cannot stat 'runs/train/exp/weights/best.pt': No such file or directory


In [ ]:
import torch
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('Using GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device('cpu')
    print('Using CPU')


Using CPU
